# Practica 3

Esta tercera practica consiste en obtener información simple y descriptiva que de indicios sobre los patrones que tiene nuestro problema. La idea es responder preguntas simples que ayuden posteriormente a formular nuestras hipótesis para futuras prácticas.

En la practica anterior se estructuro los datos de acuerdo a las distintas combinaciones de parámetros del diseño de experimentos para cada método de solución, sin embargo, para facilitarnos la tarea se trabaja con los datos crudos.

In [1]:
import pandas as pd
pd.set_option('max_rows', 10)
datos1=pd.read_csv('../mlp_datos.csv')
datos2=pd.read_csv('../mlp_datos2.csv')
datos=pd.concat([datos1,datos2])
datos=datos.rename(columns = {'trainig_time':'training_time'})
adam=datos.loc[datos.solver=='adam',['partition','layer_num','neurona_num','activation','alpha',\
                                    'learning_rate_init','train_accuracy','training_time','test_accuracy']]
lbfgs=datos.loc[datos.solver=='lbfgs',['partition','layer_num','neurona_num','activation','alpha',\
                                            'train_accuracy','training_time','test_accuracy']]
adam.to_csv('adam_crudo.csv')
lbfgs.to_csv('lbfgs_crudo.csv')

La primera pregunta es comparar ambos solver comparando las medias de los rendimientos en la fase de prueba, tomando como variable el porcentaje de datos para entrenar. Para esta pregunta propondremos que el solver `adam` es mejor que  el `lbfgs`.

In [2]:
for i in lbfgs.partition.unique():
    print(adam.test_accuracy[adam.partition==i].mean()>lbfgs.test_accuracy[lbfgs.partition==i].mean())

False
False
False
False
False
False
False
False
False


Al parecer el solver `lbfgs` es mejor que el `adam` en cada uno de los porcentajes de entrenamiento. Sin embargo, pudiera ser que bajo una combinación de los demás factores el solver `adam` sea mejor que el `lbfgs`. Para responder lo anterior veremos si el máximo del `Adam` es mejor que el de `lbfgs`, de nueva cuenta tomando como variable el porcentaje de datos para entrenar.

In [3]:
for i in lbfgs.partition.unique():
    print(adam.test_accuracy[adam.partition==i].max()>lbfgs.test_accuracy[lbfgs.partition==i].max())

False
False
False
False
False
False
False
False
False


Esto nos indica de una forma burda que el solver `lbfgs` es mejor que el `Adam`, sin embargo, es necesario realizar una prueba de hipótesis para confirmar tal sospecha. Esto nos indica de una forma burda que el solver `lbfgs` es mejor que el `Adam`, sin embargo, es necesario realizar una prueba de hipótesis para confirmar tal sospecha. Solo para realizar un análisis mas a profundidad dejaremos de lado los datos del solver `adam` y nos enfocaremos solo en los de `lbfgs`. Ahora nos enfocaremos en ver como afecta el numero de capas en el rendimiento de prueba, para ello compararemos cada nivel del factor número de capas con el anterior.

In [4]:
lbfgs.layer_num.unique()

array([ 1,  4,  7, 10], dtype=int64)

In [5]:
layers=lbfgs.layer_num.unique()[:]
for i in range(1,len(layers)):
    print(layers[i],layers[i-1])
    print(lbfgs.test_accuracy[lbfgs.layer_num==layers[i]].mean()>lbfgs.test_accuracy[lbfgs.layer_num==layers[i-1]].mean())

4 1
False
7 4
False
10 7
False


Aparentemente a medida que se aumenta el numero de capas el rendimiento promedio de prueba disminuye, pudiera ser que hay una correlación entre estas dos variables o que los niveles de los factores aun no revisados estén jugando un papel muy importante. Antes de avanzar es buena idea observar cuales son los puntos máximos y mínimos, además de la varianza.

In [6]:
for i in lbfgs.layer_num.unique():
    print('capa ',i,' min=',lbfgs.test_accuracy[lbfgs.layer_num==i].min(),'prom=',\
          lbfgs.test_accuracy[lbfgs.layer_num==i].mean(),'max=',lbfgs.test_accuracy[lbfgs.layer_num==i].max(),\
          'var=',lbfgs.test_accuracy[lbfgs.layer_num==i].var())

capa  1  min= 0.7417 prom= 0.9299193055555547 max= 0.9653 var= 0.0006301311741354017
capa  4  min= 0.0274 prom= 0.381392824074076 max= 0.9629 var= 0.15545136527965406
capa  7  min= 0.0576 prom= 0.3773751855287596 max= 0.9608 var= 0.15432714246052134
capa  10  min= 0.0892 prom= 0.37623732590529585 max= 0.9586 var= 0.15297421632094566


Lo anterior deja mas que claro el **impacto que juegan los demás factores**, porque a pesar de que con una capa se obtiene un conjunto de rendimientos de prueba que están mas “compactos” y cuyo mínimo supera por mucho a los mínimos de los otros niveles del factor numero de capas, se observa que los máximos en cada nivel son muy parecidos, incluso es difícil ver si uno es mejor que los demás. De nuevo, para seguir adelante supondremos que los mejores resultados se obtienen utilizando una cada y revisaremos como afecta el numero de neuronas, de mismo modo compararemos el nivel actual con el anterior.

In [7]:
for j in lbfgs.partition.unique():
    print('particion ', j)
    for i in lbfgs.neurona_num.unique():
        print(lbfgs.test_accuracy[(lbfgs.neurona_num==i) & (lbfgs.layer_num==1) & (lbfgs.partition==j)].mean())

particion  10
0.8664316666666668
0.9046850000000001
0.9146433333333331
0.9190033333333332
particion  20
0.8886833333333334
0.9214600000000003
0.9298383333333331
0.9341849999999998
particion  30
0.8985716666666667
0.9284849999999998
0.9369866666666666
0.9410249999999993
particion  40
0.902971666666667
0.9333900000000003
0.9416150000000003
0.9451633333333334
particion  50
0.9075533333333335
0.9370133333333331
0.944965
0.9487233333333335
particion  60
0.9104550000000005
0.9391883333333332
0.9468000000000002
0.9508516666666668
particion  70
0.9107966666666667
0.9409283333333333
0.9491666666666663
0.9530449999999999
particion  80
0.9139416666666669
0.9429116666666665
0.9508983333333335
0.9547266666666666
particion  90
0.9162833333333332
0.9443166666666666
0.9518366666666666
0.9555550000000002


Hasta aquí parece que los mejores resultados se obtienen utilizando el 90% de los datos con una capa de 210 neuronas. Además, que no parece importar que porcentaje tomemos para entrenar el rendimiento se mejora a medida que se aumenta el número de neuronas. El siguiente paso seria ver que efecto tiene la función de activación a elegir sobre el rendimiento de prueba.

In [8]:
print(lbfgs.activation.unique())
for j in lbfgs.activation.unique():
    print(lbfgs.test_accuracy[(lbfgs.neurona_num==210) & (lbfgs.layer_num==1) & (lbfgs.partition==90) & (lbfgs.activation==j)].mean())

['logistic' 'tanh']
0.95272
0.95839


Están pruebas simples indican que utilizar la función de activación **tangente hiperbólica es mejor** es mejor que la sigmoide, pero es necesario realizar una prueba de hipótesis ya que los promedios del rendimiento están muy cercanos. Por ultimo, falta checar que efecto tiene el factor de penalización (Alpha).

In [9]:
print(lbfgs.alpha.unique())
for j in lbfgs.alpha.unique():
    print(lbfgs.test_accuracy[(lbfgs.activation=='tanh') & (lbfgs.neurona_num==210) & (lbfgs.layer_num==1) & (lbfgs.partition==90) & (lbfgs.alpha==j)].mean())

[1.e-04 1.e+00 1.e+04]
0.9625700000000001
0.9620299999999998
0.9505699999999999


Todo esto nos lleva a las siguientes suposiciones, el solver `lbfgs` es mejor que el `Adam`, a medida que se aumenta el porcentaje de datos para entrenar es mejor, no es necesario utilizar mas de una capa, se debe tener mucho cuidado con la función de activación al igual que con el coeficiente de penalización. Todo esto es de gran ayuda para declarar las futuras hipótesis de prueba.